In [30]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import numpy as np
import random
import copy
import math
import matplotlib.pyplot as plt
import argparse
from collections import OrderedDict
from tqdm import tqdm
from preprocess_dataset import ProcessedDataLoader

import subprocess
from train import load_cifar10_data, validate, make_rank_k_model
device = "cpu"


In [31]:
def eval_model(model, model_name, data_aug=1):
    train_dataset = load_cifar10_data('train', './datasets',1)
    val_dataset = load_cifar10_data('val', './datasets',1)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=1, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=1, pin_memory=True)

    criterion = nn.CrossEntropyLoss().to(device)

    train_acc, train_loss = validate(model, device, train_loader, criterion)
    val_acc, val_loss = validate(model, device, val_loader, criterion)

    print(f'=================== Summary for model {model_name}===================\n',
          f'Training loss: {train_loss:.3f}   Validation loss {val_loss:.3f}   ',
          f'Training accuracy: {train_acc:.3f}   Validation accuracy: {val_acc:.3f}\n')

    return train_acc, val_acc, train_loss, val_loss

In [33]:
import os
import torch

# Define the root directory where your models are stored
# make an array with 4 file paths; collect val acc and values for k, save in an array
root_dirs = ['/home/andrew/andrew/89_FP/cs89finalproject/models/no_regularization/data_aug/nunits=256', "/home/andrew/andrew/89_FP/cs89finalproject/models/no_regularization/data_aug/nunits=1024",
             "/home/andrew/andrew/89_FP/cs89finalproject/models/no_regularization/no_data_aug/nunits=256", "/home/andrew/andrew/89_FP/cs89finalproject/models/no_regularization/no_data_aug/nunits=1024"]

all_val_accs = [] 
all_rank_constraints = []

# Walk through the directory
for root_dir in root_dirs:
    val_accs = []
    rank_constraints = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.pt'):
                # Construct the full path to the file
                file_path = os.path.join(dirpath, filename)
                model_obj = torch.load(file_path, map_location=torch.device('cpu'))

                # for the hps that were saved in a dumb way

                # need to check how we saved the hps.
                if "model state" in model_obj.keys():
                    weights = model_obj['model state']
                    congif = model_obj['hps']
                    
                    nchannels = congif.nchannels
                    nunits = congif.nunits
                    nclasses = congif.nclasses
                    nlayers = congif.nlayers
                    rank_constraint = congif.rank_constraint
                    data_aug=True
                else:
                    weights = model_obj['model']
                    nchannels = model_obj['args']['nchannels'] 
                    nunits = model_obj['args']['nunits'] 
                    nclasses = model_obj['args']['nclasses']
                    nlayers = model_obj['args']['nlayers'] 
                    rank_constraint = model_obj['args']['rank_constraint'] 
                    data_aug=True

                # rank_constraint = model_obj['args']['rank_constraint']

                model = make_rank_k_model(nchannels=nchannels, nunits=nunits, nclasses=nclasses, nlayers=nlayers, k=rank_constraint)
                model.load_state_dict(weights)

                train_acc, val_acc, train_loss, val_loss = eval_model(model, f"Model with each weight layer having rank at most {rank_constraint} during training")
                

                print(f"Model with rank constraint= {rank_constraint} got val acc of {val_acc}")
                val_accs.append(val_acc)
                rank_constraints.append(rank_constraint)
    all_val_accs.append(val_accs)
    all_rank_constraints.append(rank_constraints)
                


Data augmentation on the train set: TRUE
Files already downloaded and verified
Data augmentation on the val set: TRUE
Files already downloaded and verified
=================== Summary for model Model with each weight layer having rank at most 9 during training===================
 Training loss: 1.817   Validation loss 1.615    Training accuracy: 0.344   Validation accuracy: 0.421

Model with rank constraint= 9 got val acc of 0.4208
Data augmentation on the train set: TRUE
Files already downloaded and verified
Data augmentation on the val set: TRUE
Files already downloaded and verified
=================== Summary for model Model with each weight layer having rank at most 220 during training===================
 Training loss: 1.668   Validation loss 1.383    Training accuracy: 0.412   Validation accuracy: 0.510

Model with rank constraint= 220 got val acc of 0.5098
Data augmentation on the train set: TRUE
Files already downloaded and verified
Data augmentation on the val set: TRUE
Files 